In [1]:
# %matplotlib
# !git reset master --hard && git pull

In [2]:
# !wget http://www.openslr.org/resources/12/dev-clean.tar.gz
# !tar -zxf dev-clean.tar.gz

In [3]:
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/download_ami.sh
# !mkdir ami
# !bash download_ami.sh ami

In [4]:
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/AMI/MixHeadset.development.rttm
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/AMI/MixHeadset.test.rttm
# !wget https://raw.githubusercontent.com/pyannote/pyannote-audio/develop/tutorials/data_preparation/AMI/MixHeadset.train.rttm

In [5]:
import malaya_speech.train as train
import numpy as np
import malaya_speech
from tqdm import tqdm
import random

np.seterr(divide='raise', invalid='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [6]:
import librosa

def random_stretch(samples, low = 0.5, high = 1.3):
    input_length = len(samples)
    stretching = samples.copy()
    random_stretch = np.random.uniform(low = low, high = high)
    stretching = librosa.effects.time_stretch(
        stretching.astype('float'), random_stretch
    )
    return stretching

def random_pitch(samples, low = 0.5, high = 1.0):
    y_pitch_speed = samples.copy()
    length_change = np.random.uniform(low = low, high = high)
    speed_fac = 1.0 / length_change
    tmp = np.interp(
        np.arange(0, len(y_pitch_speed), speed_fac),
        np.arange(0, len(y_pitch_speed)),
        y_pitch_speed,
    )
    minlen = min(y_pitch_speed.shape[0], tmp.shape[0])
    y_pitch_speed *= 0
    y_pitch_speed[:minlen] = tmp[:minlen]
    return y_pitch_speed

In [7]:
import pandas as pd
import random
from tqdm import tqdm
from glob import glob

maxlen = 0.3
selected_frames = [30, 90]
functions = [random_pitch, random_stretch]

In [8]:
noises_files = glob('../noise/noise/*.wav')
noises = [malaya_speech.astype.int_to_float(malaya_speech.utils.read.wav(n)[0]) for n in tqdm(noises_files)]
noises = [n for n in noises]
len(noises)

100%|██████████| 182/182 [00:37<00:00,  4.86it/s]


182

In [9]:
def generator(sr = 16000):
        
    for i in tqdm(range(len(noises))):
        print(noises_files[i])
        x, y = [], []
        
        fs = malaya_speech.utils.generator.frames(noises[i], int(maxlen * 100), sr, False)
        x.extend([f.array for f in fs])
        y.extend([0] * len(fs))
        
        for s in selected_frames:
            fs = malaya_speech.utils.generator.frames(noises[i], s, sr, False)
            x.extend([f.array for f in fs])
            y.extend([0] * len(fs))
            
        print(len(x))
            
#         for k in range(len(x)):
#             for _ in range(random.randint(0, 3)):
#                 for f in functions:
#                     x.append(f(x[k]))
#                     y.append(y[k])
                    
#         print(len(x))
                    
        for k in range(len(x)):

            yield {
                'waveforms': x[k].tolist(),
                'targets': [int(y[k])],
            }
            
        
generator = generator()

In [10]:
import os
import tensorflow as tf

os.system('rm noise/data/*')
DATA_DIR = os.path.expanduser('noise/data')
tf.gfile.MakeDirs(DATA_DIR)

In [11]:
shards = [{'split': 'train', 'shards': 200}, {'split': 'dev', 'shards': 1}]
train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'vad')

  0%|          | 0/182 [00:00<?, ?it/s]

../noise/noise/TOP-10-INTRO-SONGSBest-Intro-Music-2018-(mp3cut.net)-(10).wav
763

INFO:tensorflow:Generating case 0.


  1%|          | 1/182 [00:00<00:22,  8.01it/s]

../noise/noise/n72.wav
310
../noise/noise/Wind-Mic-Distortion-Violent-11.wav
749


  2%|▏         | 3/182 [00:00<00:20,  8.76it/s]

...

 98%|█████████▊| 179/182 [04:28<00:02,  1.12it/s]

../noise/noise/Wind-Mic-Distortion-Violent-4.wav
826
../noise/noise/n15.wav
175
../noise/noise/n71.wav
149
../noise/noise/TOP-10-INTRO-SONGSBest-Intro-Music-2018-(mp3cut.net)-(12).wav
826


100%|██████████| 182/182 [04:28<00:00,  1.48s/it]


INFO:tensorflow:Generated 1526821 Examples


INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`

INFO:tensorflow:Data shuffled.
